# Word2Vec Encoding of Champions

1. Import modules and load data
2. Create word2vec training set of champ pairs
3. Convert to OHE (and possible downsample for memory reasons)
4. Train neural net
5. Get weights from neural net
6. Encode champs with weights from neural net and train neural net to predict match outcomes

### 1. Import modules and load data

In [1]:
import keras
from keras.layers import Dense
from keras.models import Sequential
import pandas as pd
import numpy as np
import time
import matplotlib.pyplot as plt
import os
import sys
sys.path.append('../src/')
# Project modules
import get_modeling_data
import model_evaluation.model_performance_functions as mpf
import features.win_rates as wr
import data_constants as dc
import warnings
warnings.filterwarnings('ignore')

C:\Users\Albert\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [3]:
train = get_modeling_data.get_train()
validation = get_modeling_data.get_validation()
train = train.fillna(0)
validation = validation.fillna(0)

### 2. Create word2vec training set of champ pairs

In [4]:
word2vec_train = pd.DataFrame({'input': [], 'output': []})
team_100_cols = ['100_TOP_SOLO', '100_MIDDLE_SOLO', '100_JUNGLE_NONE', '100_BOTTOM_DUO_CARRY', '100_BOTTOM_DUO_SUPPORT']
word2vec_temp = pd.DataFrame({'input': [], 'output': []})
blue_wins = train[train['team_100_win'] == 1]
for col1 in team_100_cols:
    for col2 in team_100_cols:
        if col1 != col2:
            word2vec_temp['input'] = blue_wins[col1]
            word2vec_temp['output'] = blue_wins[col2]
            word2vec_train = word2vec_train.append(word2vec_temp, ignore_index=True)

team_200_cols = ['200_TOP_SOLO', '200_MIDDLE_SOLO', '200_JUNGLE_NONE', '200_BOTTOM_DUO_CARRY', '200_BOTTOM_DUO_SUPPORT']
red_wins = train[train['team_100_win'] == 0]
for col1 in team_200_cols:
    for col2 in team_200_cols:
        if col1 != col2:
            word2vec_temp['input'] = blue_wins[col1]
            word2vec_temp['output'] = blue_wins[col2]
            word2vec_train = word2vec_train.append(word2vec_temp, ignore_index=True)
print(word2vec_train.shape)
print(word2vec_train.head())

(3386320, 2)
  input output
0  Jarv   Kata
1  Shac   Malz
2  Kled   Twis
3  Pant   Morg
4  Morg   Anni


### 3. Convert to OHE (and possible downsample for memory reasons)

In [13]:
w2v_ts = word2vec_train.sample(500000)

In [14]:
champs = dc.get_champs_four_letters()
for champ in champs:
    w2v_ts[champ + '_in'] = np.where(w2v_ts['input'] == champ, 1, 0)
    w2v_ts[champ + '_out'] = np.where(w2v_ts['output'] == champ, 1, 0)

### 4. Train neural net